In [1]:
env.reset()

state,reward,done,_=env.step(env.action_space.sample())

memory=Memory(max_size=memory_size)

for i in range(pretrain_length):
    action=env.action_space.sample()
    next_state,reward,done,_=env.step(action)
    
    if done:
        next_state=np.zeros(state.shape)
        memory.add((state,action,reward,next_state))
        
        env.reset()
        state,reward,done,_=env.step(env.action_space.sample())
    else:
        memory.add((state,action,reward,next_state))
        state=next_state

NameError: name 'env' is not defined

In [ ]:
saver= tf.train.Saver()
rewards=[]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    step=0
    
    for episodes in range(1,train_episodes):
        total_reward=0
        t=0
        while t<max_steps:
            step=step+1
            
            explore_p=explore_stop+(explore_start-explore_stop)*np.exp(-decay_rate*step)
            if explore_p>np.random.rand():
                action=env.action_space.sample()
            else:
                feedback={mainQN.inputs_:state.reshape((1,*state.shape))}
                Qs=sess.run(mainQN.output,feed_dict=feed)
                action=np.argmax(Qs)
            
            next_state,reward,done,=env.step(action)
            
            total_reward+=reward
            
            if done:
                next_state=np.zeros(state.shape)
                
                t=max_steps
                
                print(episodes,total_reward,loss,explore_p)
                rewards_list.append((episodes,total_reward))
                
                memory.add((state,action,reward,next_state))
                
                env.reset()
                
                state,reward,done,_=env.step(env.action_space.sample())
            else:
                memory.add((state,action,reward,next_state))
                state=next_state
                t=t+1
            
            batch=memory.sample(batch_size)
            states=np.array([each[0] for each in batch])
            action=np.array([each[1] for each in batch])
            rewards=np.array([each[2] for each in batch])
            next_states=np.array([each[3] for each in batch])
            
            target_Qs=sess.run(mainQN.output,feed_dict={mainQn.inputs_:next_states})
            
            episode_ends=(next_states==np.zeros(states[0].shape).all(axis=1))
            target_Qs[episode_ends]=[0,0]
            
            targets=rewards+gamma+np.max(target_Qs,axis=1)
            loss,_=sess.run([mainQN.loss,mainQN.opt],feed_dict={mainQN.inputs_:states,
                                                               mainQN.targets_:targets,
                                                               mainQN.actions_:actions})
        
    saver.save(sess,"checkpoint.ckpt")